In [1]:
# Imports
%load_ext autoreload
%autoreload 2

import re
import yaml
import numpy as np
import pandas as pd

import lts_functions as lts
import LTS_OSM

# Using jupyter notebooks with a virtual environment
# https://anbasile.github.io/posts/2017-06-25-jupyter-venv/
# https://stackoverflow.com/questions/58119823/jupyter-notebooks-in-visual-studio-code-does-not-use-the-active-virtual-environm
# ipython kernel install --user --name=.venv

In [2]:
# Settings
region = 'Cambridge'
unit = 'english'

def read_tables():
    with open('config/tables.yml', 'r') as yml_file:
        tables = yaml.safe_load(yml_file)
    return tables

def read_rating():
    with open('config/rating_dict.yml', 'r') as yml_file:
        rating_dict = yaml.safe_load(yml_file)
    return rating_dict

rating_dict = read_rating()
tables = read_tables()

In [3]:
# Load Data
gdfNodes, gdfEdges = LTS_OSM.download_data(region)

Loading saved graph for Cambridge
gdf_edges.shape=(104947, 191)
gdf_nodes.shape=(48058, 6)


In [48]:
def apply_rules(gdf_edges, rating_dict, prefix):
    rules = {k:v for (k,v) in rating_dict.items() if prefix in k}
        
    for key, value in rules.items():
        # Check rules in order, once something has been updated, leave it be
        # FIXME gracefully handle if condition is not found
        # FIXME need to handle single sided tags so that can include both sides in outputs
        # print(key)
        gdf_filter = gdf_edges.eval(f"{value['condition']} & (`{prefix}_condition` == 'default')")
        gdf_edges.loc[gdf_filter, prefix] = value[prefix]
        gdf_edges.loc[gdf_filter, f'{prefix}_rule_num'] = key
        gdf_edges.loc[gdf_filter, f'{prefix}_rule'] = value['rule_message']
        gdf_edges.loc[gdf_filter, f'{prefix}_condition'] = value['condition']
        if 'LTS' in value:
            gdf_edges.loc[gdf_filter, 'LTS'] = value['LTS']

    # Save memory by setting as category, need to set categories first
    for col in [
        # prefix,
        f'{prefix}_rule_num',
        f'{prefix}_condition', 
        f'{prefix}_rule',
        ]:

        gdf_edges[col] = gdf_edges[col].astype('category')

    return gdf_edges

In [49]:
def biking_permitted(gdf_edges, rating_dict):
    prefix = 'biking_permitted'
    defaultRule = f'{prefix}0'

    gdf_edges[prefix] = 'yes'
    gdf_edges[f'{prefix}_rule_num'] = defaultRule
    gdf_edges[f'{prefix}_rule'] = 'Assume biking permitted'
    gdf_edges[f'{prefix}_condition'] = 'default'

    gdf_edges = apply_rules(gdf_edges, rating_dict, prefix)

    return gdf_edges

rating_dict = read_rating()
gdf_biking_permitted = biking_permitted(gdfEdges.copy(), rating_dict)

rules_used = gdf_biking_permitted['biking_permitted_rule_num'].unique().tolist()
rules_used = [int(s[16:]) for s in rules_used]
rules_used.sort()
print(rules_used)

[0, 1, 5, 6]


In [50]:
# get the columns that start with 'cycleway'
cycleway_tags = gdfEdges.columns[gdfEdges.columns.str.contains('cycleway')]
# print(cycleway_tags)

for tag in cycleway_tags.sort_values():
    print(tag, gdfEdges[tag].unique())

check_date:cycleway [nan '2022-02-18' '2021-09-25' '2023-06-09']
cycleway [nan 'shared_lane' 'no' 'lane' 'separate' 'crossing' 'shared' 'track'
 'construction']
cycleway:both [nan 'no' 'shared_lane' 'separate' 'lane' 'share_busway']
cycleway:both:buffer [nan 'no' 'yes']
cycleway:both:lane [nan 'pictogram' 'exclusive' 'advisory']
cycleway:buffer [nan "2'"]
cycleway:lanes:backward [nan 'none|none|lane']
cycleway:left [nan 'shared_lane' 'no' 'separate' 'track' 'lane' 'share_busway']
cycleway:left:buffer [nan 'yes']
cycleway:left:lane [nan 'exclusive' 'pictogram']
cycleway:left:oneway [nan '-1' 'no']
cycleway:left:separation [nan 'buffer' 'flex_post']
cycleway:right [nan 'lane' 'separate' 'no' 'track' 'shared_lane' 'shoulder'
 'share_busway' 'buffered_lane']
cycleway:right:buffer [nan 'yes']
cycleway:right:lane [nan 'exclusive' 'pictogram' 'advisory']
cycleway:right:oneway [nan 'no']
cycleway:right:separation [nan 'flex_post']
cycleway:surface [nan 'asphalt']
cycleway:width [nan "5'" '6\'0

In [51]:
def is_separated_path(gdf_edges, rating_dict):   
    prefix = 'bike_lane_separation'
    defaultRule = f'{prefix}0'

    gdf_edges[prefix] = 'no'
    gdf_edges[f'{prefix}_rule_num'] = defaultRule
    gdf_edges[f'{prefix}_rule'] = 'Assume no bike lane separation'
    gdf_edges[f'{prefix}_condition'] = 'default'

    # get the columns that start with 'cycleway'
    # tags = gdf_edges.columns[gdf_edges.columns.str.contains('cycleway')]
    # for tag in tags:
    #     print(tag, gdfEdges[tag].unique())

    gdf_edges = apply_rules(gdf_edges, rating_dict, prefix)

    return gdf_edges

rating_dict = read_rating()
gdf_separated_edges = is_separated_path(gdf_biking_permitted.copy(), rating_dict)

rules_used = gdf_separated_edges['bike_lane_separation_rule_num'].unique().tolist()
rules_used = [int(s[20:]) for s in rules_used]
rules_used.sort()
print(rules_used)

[0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11]


In [52]:
# get the columns that start with 'cycleway'
cycleway_tags = gdfEdges.columns[gdfEdges.columns.str.contains('cycleway')]
# print(cycleway_tags)

for tag in cycleway_tags.sort_values():
    print(tag, gdfEdges[tag].unique())

# print('shoulder:access:bicycle', gdfEdges['shoulder:access:bicycle'].unique())

check_date:cycleway [nan '2022-02-18' '2021-09-25' '2023-06-09']
cycleway [nan 'shared_lane' 'no' 'lane' 'separate' 'crossing' 'shared' 'track'
 'construction']
cycleway:both [nan 'no' 'shared_lane' 'separate' 'lane' 'share_busway']
cycleway:both:buffer [nan 'no' 'yes']
cycleway:both:lane [nan 'pictogram' 'exclusive' 'advisory']
cycleway:buffer [nan "2'"]
cycleway:lanes:backward [nan 'none|none|lane']
cycleway:left [nan 'shared_lane' 'no' 'separate' 'track' 'lane' 'share_busway']
cycleway:left:buffer [nan 'yes']
cycleway:left:lane [nan 'exclusive' 'pictogram']
cycleway:left:oneway [nan '-1' 'no']
cycleway:left:separation [nan 'buffer' 'flex_post']
cycleway:right [nan 'lane' 'separate' 'no' 'track' 'shared_lane' 'shoulder'
 'share_busway' 'buffered_lane']
cycleway:right:buffer [nan 'yes']
cycleway:right:lane [nan 'exclusive' 'pictogram' 'advisory']
cycleway:right:oneway [nan 'no']
cycleway:right:separation [nan 'flex_post']
cycleway:surface [nan 'asphalt']
cycleway:width [nan "5'" '6\'0

In [53]:
def is_bike_lane(gdf_edges):
    """
    Check if there's a bike lane, use road features to assign LTS
    """
    # get the columns that start with 'cycleway'
    # tags = gdf_edges.columns[gdf_edges.columns.str.contains('cycleway')]
    # for tag in tags:
    #     print(tag, gdfEdges[tag].unique())

    prefix = 'bike_lane_exist'
    defaultRule = f'{prefix}0'

    gdf_edges[prefix] = 'no'
    gdf_edges[f'{prefix}_rule_num'] = defaultRule
    gdf_edges[f'{prefix}_rule'] = 'Assume no bike lane'
    gdf_edges[f'{prefix}_condition'] = 'default'

    gdf_edges = apply_rules(gdf_edges, rating_dict, prefix)

    return gdf_edges

rating_dict = read_rating()
gdf_bike_lanes = is_bike_lane(gdf_separated_edges.copy())

rules_used = gdf_bike_lanes['bike_lane_exist_rule_num'].unique().tolist()
rules_used = [int(s[15:]) for s in rules_used]
rules_used.sort()
print(rules_used)


[0, 1, 2, 3, 4]


In [54]:
tags = gdfEdges.columns[gdfEdges.columns.str.contains('parking')]
for tag in tags.sort_values():
    print(tag, gdfEdges[tag].unique())

parking:both [nan 'no' 'lane' 'separate' 'street_side']
parking:both:orientation [nan 'parallel']
parking:condition:both [nan 'no_parking' 'no_stopping' 'ticket;residents']
parking:condition:left [nan 'no_parking' 'residents' 'ticket;residents']
parking:condition:right [nan 'no_parking' 'residents']
parking:lane:both [nan 'parallel' 'no_stopping' 'no']
parking:lane:both:parallel [nan 'on_street']
parking:lane:left [nan 'parallel' 'no_stopping' 'no']
parking:lane:left:parallel [nan 'on_street']
parking:lane:right [nan 'no_stopping' 'parallel' 'no']
parking:lane:right:parallel [nan 'on_street' 'painted_area_only']
parking:left [nan 'no' 'lane' 'separate' 'street_side']
parking:left:orientation [nan 'parallel']
parking:left:restriction [nan 'no_stopping']
parking:right [nan 'lane' 'no' 'separate' 'street_side']
parking:right:access [nan 'permit']
parking:right:fee [nan 'yes']
parking:right:orientation [nan 'parallel']


In [55]:
def parking_present(gdf_edges):
    """
    Detect where parking is and isn't allowed.
    """
    # tags = gdfEdges.columns[gdfEdges.columns.str.contains('parking')]
    # for tag in tags.sort_values():
    #     print(tag, gdfEdges[tag].unique())

    prefix = 'parking'
    defaultRule = f'{prefix}0'

    gdf_edges[prefix] = 'yes'
    gdf_edges[f'{prefix}_rule_num'] = defaultRule
    gdf_edges[f'{prefix}_rule'] = 'Assume street parking is allowed on both sides'
    gdf_edges[f'{prefix}_condition'] = 'default'

    gdf_edges = apply_rules(gdf_edges, rating_dict, prefix)


    gdf_edges['width_parking'] = 0
    gdf_edges.loc[gdf_edges[prefix]=='yes', 'width_parking'] = 8.5 # ft

    return gdf_edges

rating_dict = read_rating()
gdf_parking = parking_present(gdf_bike_lanes.copy())

rules_used = gdf_parking['parking_rule_num'].unique().tolist()
rules_used = [int(s[7:]) for s in rules_used]
rules_used.sort()
print(rules_used)

[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 21, 22, 23, 24, 26, 28, 29, 30, 41, 42, 44, 50]


C:\Users\Scott\AppData\Local\Temp\ipykernel_5588\2975178244.py:21: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '8.5' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  gdf_edges.loc[gdf_edges[prefix]=='yes', 'width_parking'] = 8.5 # ft


In [56]:
tags = gdfEdges.columns[gdfEdges.columns.str.contains('speed')]
for tag in tags.sort_values():
    print(tag, gdfEdges[tag].unique())

print('\n---\n')
tags = gdfEdges.columns[gdfEdges.columns.str.contains('highway')]
for tag in tags.sort_values():
    print(tag, gdfEdges[tag].unique())

maxspeed [nan '25 mph' '30 mph' '20 mph' '15 mph' '35 mph' '25' '40 mph' '20'
 '55 mph' '10 mph' '5 mph']
maxspeed:advisory [nan '5 mph']
maxspeed:type [nan 'US:urban' 'sign']
source:maxspeed [nan 'massgis']

---

area:highway [nan 'traffic_island' 'yes']
construction:highway [nan 'cycleway']
highway ['footway' 'residential' 'secondary' 'unclassified' 'trunk' 'service'
 'tertiary' 'primary' 'cycleway' 'path' 'primary_link' 'pedestrian'
 'trunk_link' 'tertiary_link' 'secondary_link' 'construction'
 'living_street' 'busway' 'track']
note:highway [nan 'busway']


In [57]:
print(gdfEdges[gdfEdges['highway'] == 'primary']['maxspeed'].unique())

['25 mph' nan '35 mph' '25' '20 mph' '40 mph' '30 mph']


In [58]:
def get_prevailing_speed(gdf_edges):
    """
    Get the speed limit for ways
    If not available, make assumptions based on road type
    This errs on the high end of assumptions
    """
    prefix = 'speed'
    speedRules = {k:v for (k,v) in rating_dict.items() if prefix in k}
    defaultRule = f'{prefix}0'

    # FIXME if change to apply assumed values first then replace with OSM data, can use common function
    gdf_edges['speed'] = gdf_edges['maxspeed'] 
    gdf_edges['speed'] = gdf_edges['speed'].fillna(0)
    gdf_edges['speed_rule_num'] = defaultRule
    gdf_edges['speed_rule'] = 'Use signed speed limits.'
    gdf_edges['speed_condition'] = 'default'

    for key, value in speedRules.items():
        # Check rules in order, once something has been updated, leave it be
        gdf_filter = gdf_edges.eval(f"{value['condition']} & (`speed` == 0)")
        gdf_edges.loc[gdf_filter, 'speed'] = value['speed']
        gdf_edges.loc[gdf_filter, 'speed_rule_num'] = key
        gdf_edges.loc[gdf_filter, 'speed_rule'] = value['rule_message']
        gdf_edges.loc[gdf_filter, 'speed_condition'] = value['condition']

    # If mph
    if gdf_edges[gdf_edges['speed'].astype(str).str.contains('mph')].shape[0] > 0:
        mph = gdf_edges['speed'].astype(str).str.contains('mph', na=False)
        gdf_edges.loc[mph, 'speed'] = gdf_edges['speed'][mph].str.split(
            ' ', expand=True)[0].apply(lambda x: np.array(x, dtype = 'int'))

    # # if multiple speed values present, use the largest one
    # gdf_edges['maxspeed_assumed'] = gdf_edges['maxspeed_assumed'].apply(
    #     lambda x: np.array(x, dtype = 'int')).apply(lambda x: np.max(x))

    # Make sure all speeds are numbers
    gdf_edges['speed'] = gdf_edges['speed'].astype(int)
    # Save memory by setting as category, need to set categories first
    for col in ['speed_rule_num', 'speed_rule', 'speed_condition']:
        gdf_edges[col] = gdf_edges[col].astype('category')

    return gdf_edges

rating_dict = read_rating()
gdf_speed = get_prevailing_speed(gdf_parking.copy())

rules_used = gdf_speed['speed_rule_num'].unique().tolist()
rules_used = [int(s[5:]) for s in rules_used]
rules_used.sort()
print(rules_used)

print(gdf_speed['speed'].unique())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
[25 30 20 35 40 50 15  0 55 10  5]


In [59]:
print(gdfEdges['lanes'].unique())

[nan '2' '1' '3' '4' '5' '7']


In [60]:
def get_lanes(gdf_edges, default_lanes = 2):

    # make new assumed lanes column for use in calculations

    # fill na with default lanes
    # if multiple lane values present, use the largest one
    # this usually happens if multiple adjacent ways are included in the edge and 
    # there's a turning lane
    gdf_edges['lane_count'] = gdf_edges['lanes'].fillna(default_lanes).apply(
        lambda x: np.array(re.split(r'; |, |\*|\n', str(x)), dtype = 'int')).apply( 
            # Converted to a raw string to avoid 'SyntaxWarning: invalid escape sequence '\*' python re',
            # check that this is doing the right thing
            lambda x: np.max(x))
    
    gdf_edges['lane_source'] = 'OSM'
    assumed = gdf_edges['lanes'] == np.nan
    gdf_edges.loc[assumed, 'lane_source'] = 'Assumed lane count'

    return gdf_edges

rating_dict = read_rating()
gdf_lanes = get_lanes(gdf_speed.copy())

In [61]:
print(gdf_lanes['lane_count'].unique())

[2 1 3 4 5 7]


In [62]:
tags = gdfEdges.columns[gdfEdges.columns.str.contains('lane_markings')]
for tag in tags.sort_values():
    print(tag, gdfEdges[tag].unique())

lane_markings [nan 'no' 'yes']


In [63]:
# This is new.
# The newer LTS rating takes into account whether there is a centerline striped or not.
def get_centerlines(gdf_edges):

    prefix = 'centerline'
    defaultRule = f'{prefix}0'

    gdf_edges[prefix] = 'yes'
    gdf_edges[f'{prefix}_rule_num'] = defaultRule
    gdf_edges[f'{prefix}_rule'] = 'Assume centerlines.'
    gdf_edges[f'{prefix}_condition'] = 'default'

    gdf_edges = apply_rules(gdf_edges, rating_dict, prefix)

    return gdf_edges

rating_dict = read_rating()
gdf_centerlines = get_centerlines(gdf_lanes.copy())

rules_used = gdf_centerlines['centerline_rule_num'].unique().tolist()
rules_used = [int(s[10:]) for s in rules_used]
rules_used.sort()
print(rules_used)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]


In [64]:
tags = gdfEdges.columns[gdfEdges.columns.str.contains('width')]
for tag in tags.sort_values():
    print(tag, gdfEdges[tag].unique())

tags = gdfEdges.columns[gdfEdges.columns.str.contains('buffer')]
for tag in tags.sort_values():
    print(tag, gdfEdges[tag].unique())

cycleway:width [nan "5'" '6\'0"' '3\'0"']
source:width [nan 'ARCore']
width [nan '7.6' '9.5' '15.2' '21.3' '14.9' '12.2' '11.6' '30.5' '18.9' '20.7'
 '11.0' '7.3' '15.9' '9.1' '17\'0"' '11.9' '20\'0"' '17.1' '24.4' '14.0'
 '18.3' '24\'0"' '30.2' '14.6' '15.2;18.6' '13.4' '25.9' '6.7' '28.0'
 '36.6' '16.8' '3.7' '19.8' '10.4;12.2' '11.7' '4.3' '4.6' '8.5' '12.8'
 '10.4' '15.5' '13.7' '6.1' '25.6' '9.8' '4.9' '26.8' '3.0' '11.3' '5.5'
 '10.7' '19.2' '7.9' '5.8' '8.2' '10.1' '28\'0"' '20.1' '18.6' '6.4' '5.2'
 '16.5' '14.3' '17.7' '19.5' '38.1' '8.8' '22.9' '30\'0"' '16.2' '4.7'
 '16.8;15.2' '7' '2' '12.5' '46\'0"' '50\'0"' '16.1' '27\'0"' '91.5'
 '18.0' '11.6;10.4' '5' '27.4' '8\'0"' '3.3' '3' '3.5' '8\'8"' '5\'0"' '6'
 '21\'0"' "10'" '1' '2.7' '1.5' '9\'0"' '8' '10\'0"' '12\'0"' '1.4' "8'"
 '7.0' '1.8' '6\'0"' '16\'0"']
width:carriageway [nan '9' '8']
width:lanes [nan '3|3']
cycleway:both:buffer [nan 'no' 'yes']
cycleway:buffer [nan "2'"]
cycleway:left:buffer [nan 'yes']
cycleway:right:

In [65]:
def convert_feet_with_quotes(series):
    series = series.copy()
    # Calculate decimal feet and inches when each given separately
    quoteValues = series.str.contains('\'')
    meterValues = quoteValues == False

    quoteValues[quoteValues.isna()] = False
    quoteValues = quoteValues.astype(bool)

    feetinch = series[quoteValues].str.strip('"').str.split('\'', expand=True)
    feetinch.loc[feetinch[1] == '', 1] = 0
    feetinch = feetinch.apply(lambda x: np.array(x, dtype = 'int'))
    if feetinch.shape[0] > 0:
        feet = feetinch[0] + feetinch[1] / 12
        series[quoteValues] = feet

    # Use larger value if given multiple
    multiWidth = series.str.contains(';', na=False) 

    maxWidth = series[multiWidth].str.split(';', expand=True).max(axis=1)
    series[multiWidth] = maxWidth

    series = series.apply(lambda x: np.array(x, dtype = 'float'))

    # Convert (assumed) meter values to feet
    series[meterValues] = series[meterValues].astype(float) * 3.28084

    series_notes = pd.Series('No Width', index=series.index)
    series_notes[quoteValues] = 'Converted ft-in to decimal feet'
    series_notes[meterValues] = 'Converted m to feet'

    return series, series_notes

def width_ft(gdf_edges):
    '''
    Convert OSM width columns to use decimal feet
    '''
    # print('width_ft')
    gdf_edges['width_street'], gdf_edges['width_street_notes'] = convert_feet_with_quotes(gdf_edges['width'])
    # print('cycleway:width_ft')
    gdf_edges['width_bikelane'], gdf_edges['width_bikelane_notes'] = convert_feet_with_quotes(gdf_edges['cycleway:width'])

    gdf_edges['width_bikelanebuffer'], gdf_edges['width_bikelanebuffer_notes'] = convert_feet_with_quotes(gdf_edges['cycleway:buffer'])

    # FIXME make this work for asymmetric layouts
    gdf_edges['bikelane_reach'] = gdf_edges['width_bikelane'] + gdf_edges['width_parking'] + gdf_edges['width_bikelanebuffer']


    return gdf_edges

gdf_width_ft = width_ft(gdf_centerlines.copy())

In [66]:
streetWidth = gdf_width_ft['width_street'].unique()
cyclewayWidth = gdf_width_ft['width_bikelane'].unique()

streetWidth.sort()
cyclewayWidth.sort()

print(streetWidth)
print(cyclewayWidth)

[  3.28084      4.593176     4.92126      5.           5.905512
   6.           6.56168      8.           8.66666667   8.858268
   9.           9.84252     10.          10.826772    11.48294
  12.          12.139108    14.107612    15.091864    15.419948
  16.          16.076116    16.4042      17.          17.060368
  18.04462     19.028872    19.68504     20.          20.013124
  20.997376    21.          21.981628    22.96588     23.950132
  24.          24.934384    25.918636    26.24672     26.902888
  27.          27.88714     28.          28.871392    29.855644
  30.          31.16798     32.152232    33.136484    34.120736
  35.104988    36.08924     37.073492    38.057744    38.385828
  39.041996    40.026248    41.0105      41.994752    43.963256
  44.947508    45.93176     46.          46.916012    47.900264
  48.884516    49.868768    50.          50.85302     52.165356
  52.821524    53.149608    54.13386     55.118112    56.102364
  58.070868    59.05512     60.039372    

In [67]:
tags = gdfEdges.columns[gdfEdges.columns.str.contains('one')]
for tag in tags.sort_values():
    print(tag, gdfEdges[tag].unique())

tags = gdf_width_ft.columns[gdf_width_ft.columns.str.contains('parking')]
for tag in tags.sort_values():
    print(tag, gdf_width_ft[tag].unique())

cycleway:left:oneway [nan '-1' 'no']
cycleway:right:oneway [nan 'no']
oneway [False  True]
oneway:bicycle [nan 'no' 'yes']
oneway:conditional [nan '-1 @ (17:00-19:00); ' 'yes @ (07:00-09:00,15:00-19:00)']
parking ['yes' 'no' 'left:no' 'left:yes' 'right:no' 'right:yes']
parking:both [nan 'no' 'lane' 'separate' 'street_side']
parking:both:orientation [nan 'parallel']
parking:condition:both [nan 'no_parking' 'no_stopping' 'ticket;residents']
parking:condition:left [nan 'no_parking' 'residents' 'ticket;residents']
parking:condition:right [nan 'no_parking' 'residents']
parking:lane:both [nan 'parallel' 'no_stopping' 'no']
parking:lane:both:parallel [nan 'on_street']
parking:lane:left [nan 'parallel' 'no_stopping' 'no']
parking:lane:left:parallel [nan 'on_street']
parking:lane:right [nan 'no_stopping' 'parallel' 'no']
parking:lane:right:parallel [nan 'on_street' 'painted_area_only']
parking:left [nan 'no' 'lane' 'separate' 'street_side']
parking:left:orientation [nan 'parallel']
parking:left

In [69]:
# This is new.
# There are some conditions where the wideness of the street affects the rating in the new LTS ratings.
def define_narrow_wide(gdf_edges):

    gdf_edges['street_narrow_wide'] = 'wide'

    gdf_edges[(gdf_edges['oneway'] == 'True') & (gdf_edges['width_street'] < 30) & (gdf_edges['parking'] == 'yes')] = 'narrow'

    # FIXME make sure only single side has parking and not ignoring where one side is explicit yes and the other is explicit no
    gdf_edges[(gdf_edges['oneway'] == 'True') & (gdf_edges['width_street'] < 22) & (gdf_edges['parking'] == 'left:yes')] = 'narrow'
    gdf_edges[(gdf_edges['oneway'] == 'True') & (gdf_edges['width_street'] < 22) & (gdf_edges['parking'] == 'right:yes')] = 'narrow'

    gdf_edges[(gdf_edges['oneway'] == 'True') & (gdf_edges['width_street'] < 15) & (gdf_edges['parking'] == 'no')] = 'narrow'

    return gdf_edges

rating_dict = read_rating()
gdf_nw = define_narrow_wide(gdf_width_ft.copy())

In [70]:
# This is new.
def define_adt(gdf_edges):
    '''
    Add the Average Daily Traffic (ADT) value to each segment to use the right row of LTS tables.

    Use assumptions based on roadway type. 

    FUTURE: Get ADT measurements from cities or Streetlight to improve values.
    '''

    prefix = 'ADT'
    defaultRule = f'{prefix}0'

    gdf_edges[prefix] = 1500 # FIXME is this the right default?
    gdf_edges[f'{prefix}_rule_num'] = defaultRule
    gdf_edges[f'{prefix}_rule'] = 'Assume centerlines.'
    gdf_edges[f'{prefix}_condition'] = 'default'

    gdf_edges = apply_rules(gdf_edges, rating_dict, prefix)

    return gdf_edges

rating_dict = read_rating()
gdf_adt = define_adt(gdf_nw.copy())

In [71]:
print(gdf_adt['lane_count'].unique())
print(gdf_adt['speed'].unique())
print(gdf_adt['ADT'].unique())

[2 1 3 4 5 7]
[25 30 20 35 40 50 15  0 55 10  5]
[1500]


In [72]:
tags = gdf_adt.columns[gdf_adt.columns.str.contains('contra')]
for tag in tags.sort_values():
    print(tag, gdf_adt[tag].unique())

In [91]:
def evaluate_lts_table(gdf_edges, tables, tableName):
    baseName = tableName[6:]
    table = tables[tableName]

    subTables = [key for key in table.keys() if tableName in key]
    print(subTables)

    speedMin = tables['cols_speeds']['min']
    speedMax = tables['cols_speeds']['max']

    gdf_edges[f'LTS_{baseName}'] = np.nan

    conditionTable = table['conditions']

    for subTable in subTables:
        print(f'\n{subTable=}')
        # print(f'{table[subTable]['conditions']=}')
        for conditionTableName in table['conditions']:
            conditionTable = table['conditions'][conditionTableName]
            # print(conditionTable)
            for conditionName in table[subTable]['conditions']:
                bucketColumn = table['bucketColumn']
                bucketTable = table[subTable][f'table_{bucketColumn}']
                ltsSpeeds = table[subTable]['table_speed']
                for bucket, ltsSpeed in zip(bucketTable, ltsSpeeds):
                    conditionBucket = f'(`{bucketColumn}` >= {bucket[0]}) & (`{bucketColumn}` < {bucket[1]})'
                    for sMin, sMax, lts in zip(speedMin, speedMax, ltsSpeed):
                        condition = table[subTable]['conditions'][conditionName]
                        conditionSpeed = f'(`speed` > {sMin}) & (`speed` < {sMax})'
                        condition = f'{condition} & {conditionSpeed} & {conditionBucket} & {conditionTable}'
                        # print(f'\t{conditionName} | {condition}')
                        gdf_filter = gdf_edges.eval(f"{condition}")
                        gdf_edges.loc[gdf_filter, f'LTS_{baseName}'] = lts
                # gdf_edges.loc[gdf_filter, f'{prefix}_rule_num'] = key
            

    return gdf_edges

def calculate_lts(gdf_edges, tables):
    # print(tables)
    tablesList = [key for key in tables.keys() if 'table_' in key]

    for tableName in tablesList:
        gdf_edges = evaluate_lts_table(gdf_edges, tables, tableName)

    # Use the lowest calculated LTS score for a segment (in case mixed is lower than bike lane)
    gdf_edges['LTS'] = gdf_edges.loc[:, gdf_edges.columns.str.contains('LTS')].min(axis=1, skipna=True, numeric_only=True)

    return gdf_edges


rating_dict = read_rating()
tables = read_tables()
gdf_lts = calculate_lts(gdf_adt.copy(), tables)

['table_mixed_1', 'table_mixed_2', 'table_mixed_3', 'table_mixed_4', 'table_mixed_5']

subTable='table_mixed_1'

subTable='table_mixed_2'

subTable='table_mixed_3'

subTable='table_mixed_4'

subTable='table_mixed_5'
['table_bikelane_noparking_1', 'table_bikelane_noparking_2', 'table_bikelane_noparking_3']

subTable='table_bikelane_noparking_1'

subTable='table_bikelane_noparking_2'

subTable='table_bikelane_noparking_3'
['table_bikelane_yesparking_1', 'table_bikelane_yesparking_2', 'table_bikelane_yesparking_3', 'table_bikelane_yesparking_4']

subTable='table_bikelane_yesparking_1'

subTable='table_bikelane_yesparking_2'

subTable='table_bikelane_yesparking_3'

subTable='table_bikelane_yesparking_4'


In [87]:
print(f"{gdf_lts['LTS'].unique()=}")
print(f"{gdf_lts['LTS_mixed'].unique()=}")
print(f"{gdf_lts['centerline'].unique()=}")
print(f"{gdf_lts['lane_count'].unique()=}")
print(f"{gdf_lts['oneway'].unique()=}")

gdf_lts['LTS'].unique()=array([ 1.,  2., nan,  3.,  4.,  0.])
gdf_lts['LTS_mixed'].unique()=array([ 1.,  2., nan,  3.,  4.])
gdf_lts['centerline'].unique()=array(['no', 'yes'], dtype=object)
gdf_lts['lane_count'].unique()=array([2, 1, 3, 4, 5, 7])
gdf_lts['oneway'].unique()=array([False,  True])


In [88]:
gdf_lts.loc[gdf_lts.eval("(`centerline` == 'no') & (`lane_count` == 2) & (`oneway` == False) & (`speed` > 0) & (`speed` < 23.5) & (`ADT` >= 751) & (`ADT` < 1501)")].shape

(5054, 234)